<a href="https://colab.research.google.com/github/vipbaswan103/DSA/blob/master/Data_preprocess_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [283]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [284]:
ls

dictionaryEtoF.txt      dictionaryFtoE.txt.txt  PreProcessedData_1/
dictionaryEtoF.txt.txt  EM_Draft1.ipynb         PreProcessedData_Final/
dictionaryFtoE.txt      EM_FinalDraft.ipynb     Training/


In [24]:
!pip install pyproj

In [0]:
import numpy as np
import pandas as pd
# import sklearn as sk
# from sklearn.decomposition import PCA
# from geopy.distance import geodesic
# import mpu
import pyproj
# import warnings
# warnings.filterwarnings('ignore')
# except:
#     pass
# import keras
# # from keras.activation import adam
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Input
# from keras.utils import to_categorical
import glob

from datetime import datetime
from datetime import timedelta
from IPython.display import clear_output


In [0]:
path = 'Training/20190818'
all_files = glob.glob(path+ "/*.csv")
print(all_files)
clear_output()

['Training/20190818/Itriangle3_20190818_072702.csv', 'Training/20190818/Itriangle3_20190818_073001.csv', 'Training/20190818/Itriangle3_20190818_073102.csv', 'Training/20190818/Itriangle3_20190818_073601.csv', 'Training/20190818/Itriangle3_20190818_073702.csv', 'Training/20190818/Itriangle3_20190818_073201.csv', 'Training/20190818/Itriangle3_20190818_074002.csv', 'Training/20190818/Itriangle3_20190818_073802.csv', 'Training/20190818/Itriangle3_20190818_073501.csv', 'Training/20190818/Itriangle3_20190818_073401.csv', 'Training/20190818/Itriangle3_20190818_073302.csv', 'Training/20190818/Itriangle3_20190818_073901.csv', 'Training/20190818/Itriangle3_20190818_074101.csv', 'Training/20190818/Itriangle3_20190818_074201.csv', 'Training/20190818/Itriangle3_20190818_074501.csv', 'Training/20190818/Itriangle3_20190818_074401.csv', 'Training/20190818/Itriangle3_20190818_074301.csv', 'Training/20190818/Itriangle3_20190818_075101.csv', 'Training/20190818/Itriangle3_20190818_074601.csv', 'Training/2

In [0]:
li=[]
i=0
for filename in all_files :
    df = pd.read_csv(filename, index_col=None, header= None, sep=';')
    if (i%10 == 5):
      print(i)
    i = i + 1
    li.append(df)
clear_output()

In [0]:
for i in range(len(li)):
    df=li[i]
    df.rename(columns={0:'id',1: 'time', 2:'lat', 3:'long',4: 'speed',5: 'class'}, inplace=True)
    if i%50 == 25:
      print(i)
    i=i+1
clear_output()

In [0]:
uniqueID=dict()
for i in range(len(li)):
  for j in range(li[i].shape[0]):
    if(uniqueID.get(li[i].at[j,'id']) == None):
      uniqueID[li[i].at[j,'id']] = 1
    else:
      uniqueID[li[i].at[j,'id']] += 1
  if i%50 == 25:
    print(i)

clear_output()

In [0]:
dictOfList={}
for ID in uniqueID:
    dictOfList[ID] = [] #create a dictionary of lists
for j in range(len(li)):
  for k in range(li[j].shape[0]):
    tmpDict = {}
    tmpDict.update({'id':li[j].at[k,'id']})
    tmpDict.update({'time':li[j].at[k,'time']})
    tmpDict.update({'lat':li[j].at[k,'lat']})
    tmpDict.update({'long':li[j].at[k,'long']})
    tmpDict.update({'speed':li[j].at[k,'speed']})
    tmpDict.update({'class':li[j].at[k,'class']})
    dictOfList[li[j].at[k,'id']].append(tmpDict)   
  if(j%10 == 0):
    print(j)  
clear_output() 

In [0]:
preData=[]
for key in dictOfList:
  frame=pd.DataFrame(dictOfList[key])
  preData.append(frame)

In [0]:
for i in range(len(preData)):
  name = './PreProcessedData_1/DataFile_'+str(i+1)+'.csv'
  preData[i].to_csv(name)

In [0]:
path = './PreProcessedData_1'
all_files = glob.glob(path+ "/*.csv")
vehicles=[]
i = 0
for fileName in all_files:
  frame = pd.read_csv(fileName, sep = ',')
  vehicles.append(frame)
  if i%100 == 0:
    print(i)
  i += 1

clear_output()

In [0]:
# Adding new columns for speed,datetime,etc
for i in range(len(vehicles)):
    vehicles[i].insert(loc=6, column='dist', value=1.0)
    vehicles[i].insert(loc=7, column='LatLongSpeed', value=1.0)
    vehicles[i].insert(loc=8, column='acc', value=1.0)
    vehicles[i].insert(loc=9, column='datetime',value=datetime.strptime('1857-08-15 00:00:00', '%Y-%m-%d %H:%M:%S'))
    if(i%100==0):
      print(i)
clear_output()

In [0]:
#populating datetime and sorting according to time
for j in range(len(vehicles)):
    for k in range(0, vehicles[j].shape[0]):
        vehicles[j].at[k,'datetime'] = datetime.strptime(vehicles[j].at[k,'time'], '%Y-%m-%d %H:%M:%S')
    vehicles[j].sort_values(by='datetime',inplace=True,na_position='first')
    if(j%100 == 0):
      print(j)
clear_output()

In [0]:
for i in range(len(vehicles)):
  vehicles[i].drop_duplicates(subset=['datetime'], keep='first', inplace=True)

In [0]:
#Just to check whether duplicates remain or not
for i in range(len(vehicles)):
  if(any(vehicles[i]['datetime'].duplicated()) == True):
    print("Duplicate Exists in "+str(i))

In [0]:
vehicles_new=[]
for i in range(len(vehicles)):
    reset_index_df = vehicles[i].reset_index(drop=True)
    vehicles_new.append(reset_index_df[['id', 'datetime','lat','long','dist','speed','LatLongSpeed','acc','class']].copy())

In [0]:
#In case, if duplicates remain, delete them forcefully
for i in range(len(vehicles_new)):
    duplicateRowsDF = vehicles_new[i][vehicles_new[i].duplicated(subset = ['datetime'],keep = 'first')]
    print("Duplicate Rows of :"+ str(i))
    index_duplicates=list(duplicateRowsDF.index.values)
    vehicles_new[i].drop(vehicles_new[i].index[index_duplicates],inplace = True)
clear_output()

In [0]:
#Check if some dataframe is empty or not
for i in range(len(vehicles_new)):
  if(vehicles_new[i].empty):
    print("I am Empty")
clear_output()

In [0]:
for j in range(len(vehicles_new)):
    # if(not vehicles_new[j].empty):
    vehicles_new[j].at[0,'dist']=0
    vehicles_new[j].at[0,'LatLongSpeed']=0
    vehicles_new[j].at[0,'acc']=0
    for k in range(1, vehicles_new[j].shape[0]):
        geod = pyproj.Geod(ellps='WGS84')
        lat0=vehicles_new[j].at[k-1,'lat']
        lon0=vehicles_new[j].at[k-1,'long']
        lat1=vehicles_new[j].at[k,'lat']
        lon1=vehicles_new[j].at[k,'long']

        a1, a2, dist = geod.inv(lon0, lat0, lon1, lat1)
        vehicles_new[j].at[k,'dist']=dist
        
        timediff = (vehicles_new[j].at[k,'datetime']-vehicles_new[j].at[k-1,'datetime']).total_seconds()/3600
        
        
        vehicles_new[j].at[k,'LatLongSpeed']=(vehicles_new[j].at[k,'dist']-vehicles_new[j].at[k-1,'dist'])/timediff
        
        vehicles_new[j].at[k,'acc']=(vehicles_new[j].at[k,'speed']-vehicles_new[j].at[k-1,'speed'])/timediff
    
    if(j%100 == 0):
      print(j)
clear_output()

In [0]:
# To check if nan, inf or -inf exist in some entry of some row of some dataframe
for i in range(len(vehicles)):
    if(vehicles_new[i].isin([np.nan, np.inf, -np.inf]).any(1).values.any()):
        print(str(i)+" Exist")
    else:
        print(str(i)+" Not Exist")
clear_output()

In [0]:
df2=pd.DataFrame(columns=['id','speed1_avg','speed2_avg','speed1_var','speed2_var','acc_avg','acc_var','class'])
for i in range(len(vehicles_new)):
    if(not vehicles_new[j].empty):
        df2.at[i,'speed1_avg'] = vehicles_new[i]['speed'].mean()
        df2.at[i,'speed2_avg'] = vehicles_new[i]['LatLongSpeed'].mean()
        df2.at[i,'speed1_var'] = vehicles_new[i]['speed'].var()
        df2.at[i,'speed2_var'] = vehicles_new[i]['LatLongSpeed'].var()
        df2.at[i,'acc_avg'] = vehicles_new[i]['acc'].mean()
        df2.at[i,'acc_var'] = vehicles_new[i]['acc'].var()
        df2.at[i,'id']= i
        df2.at[i,'class']= vehicles_new[i]['class'][0]
    if(i%100 == 0):
      print(i)
clear_output()

In [0]:
df2.to_csv('./PreProcessedData_Final/FinalData.csv')

In [286]:
df2 = pd.read_csv('./PreProcessedData_Final/FinalData.csv')
df2['class'].unique()
df2['class'].nunique()

11

In [0]:
df3 = df2
df3 = df3.drop(columns=['class', 'id'])
df3=df3[df3.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)] 

In [0]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [289]:
for i in range(1,7):
  pca = PCA(n_components = i)
  pca.fit(df3)
  print(sum(pca.explained_variance_ratio_))

1.0000000000000009
1.0000000000000009
1.000000000000001
1.0000000000000007
1.0000000000000009
1.0


In [0]:
df4 = df2
y = df4['class']
df4=df4.drop(columns = ['id','class'])
df4 = df4.fillna(df4.mean())

In [0]:
X_tr, X_tst, y_tr, y_tst = train_test_split(df4, y, test_size=0.3)

In [292]:
#KNN Model
clf=KNeighborsClassifier()
clf.fit(X_tr, y_tr)
predicted= clf.predict(X_tst)
print(metrics.accuracy_score(y_tst, predicted))

0.742393509127789


In [293]:
#Naive Bayes Model
clf=BernoulliNB()
clf.fit(X_tr, y_tr)
predicted= clf.predict(X_tst)
print(metrics.accuracy_score(y_tst, predicted))

0.7505070993914807


In [294]:
#Space Vector Classification Model
clf=SVC()
clf.fit(X_tr, y_tr)
predicted= clf.predict(X_tst)
print(metrics.accuracy_score(y_tst, predicted))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7728194726166329


In [295]:
#Random Forest Model
clf=RandomForestClassifier()
clf.fit(X_tr, y_tr)
predicted= clf.predict(X_tst)
print(metrics.accuracy_score(y_tst, predicted))

0.9421906693711968


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [296]:
#To evaluate the model's predictions
report = classification_report(y_tst, predicted)
print(report)
matrix = confusion_matrix(y_tst, predicted)
print(matrix)

               precision    recall  f1-score   support

    Ambulance       0.95      0.97      0.96       175
          Bus       0.00      0.00      0.00         5
          Cab       0.00      0.00      0.00         1
          Car       0.95      0.98      0.97       762
     Mini_Bus       0.00      0.00      0.00         1
  Motor Cycle       1.00      0.11      0.20         9
Not available       0.00      0.00      0.00        11
      Tractor       1.00      0.50      0.67         2
        Truck       0.53      0.45      0.49        20

     accuracy                           0.94       986
    macro avg       0.49      0.33      0.36       986
 weighted avg       0.93      0.94      0.93       986

[[170   0   0   5   0   0   0   0   0]
 [  0   0   0   5   0   0   0   0   0]
 [  0   0   0   1   0   0   0   0   0]
 [  5   0   0 748   0   0   1   0   8]
 [  0   0   0   1   0   0   0   0   0]
 [  3   0   0   4   0   1   1   0   0]
 [  0   0   0  11   0   0   0   0   0]
 [  0   0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
